In [1]:
import pandas as pd
import json
import requests

In [2]:
url = " https://seeclickfix.com/api/v2/issues?min_lat=39.03211937423035&min_lng=-95.71889877319336&max_lat=39.052118348329074&max_lng=-95.6312656402588&status=open%2Cacknowledged%2Cclosed&fields%5Bissue%5D=id%2Csummary%2Cdescription%2Cstatus%2Clat%2Clng%2Caddress%2Cmedia%2Ccreated_at%2Cacknowledged_at%2Cclosed_at&page=1"

In [3]:
response = requests.get(url).json()

In [8]:
next_page = response['metadata']['pagination']['next_page_url']
next_page

'https://seeclickfix.com/api/v2/issues?fields%5Bissue%5D=id%2Csummary%2Cdescription%2Cstatus%2Clat%2Clng%2Caddress%2Cmedia%2Ccreated_at%2Cacknowledged_at%2Cclosed_at&max_lat=39.052118348329074&max_lng=-95.6312656402588&min_lat=39.03211937423035&min_lng=-95.71889877319336&page=2&status=open%2Cacknowledged%2Cclosed'

In [9]:
issues = response['issues']

In [11]:
pd.DataFrame(issues)

,id,status,summary,description,lat,lng,address,created_at,acknowledged_at,closed_at,url,media
0,11864430,Acknowledged,Other,Area filled with trash!!!,39.037521,-95.667171,"900 Se Maryland Ave Topeka 66607, United States",2022-02-11T16:25:59-05:00,2022-02-11T16:26:11-05:00,None,https://seeclickfix.com/api/v2/issues/11864430,"{'video_url': None, 'image_full': 'https://see..."
1,11854694,Closed,Trees and Tree Limbs,This branch is hanging over the sidewalk and l...,39.051680,-95.692372,"905 Sw Lincoln St Topeka 66606, United States",2022-02-09T17:37:58-05:00,2022-02-09T17:38:12-05:00,2022-02-10T09:51:49-05:00,https://seeclickfix.com/api/v2/issues/11854694,"{'video_url': None, 'image_full': 'https://see..."
2,11852869,Acknowledged,Pothole,The pothole is on 17th street just before you ...,39.037040,-95.671273,"600-698 Se 17th St Topeka, KS, 66607, USA",2022-02-09T12:43:02-05:00,2022-02-09T12:43:07-05:00,None,https://seeclickfix.com/api/v2/issues/11852869,"{'video_url': None, 'image_full': None, 'image..."
3,11852838,Acknowledged,Pothole,There is a pothole at 17th and Topeka Blvd. on...,39.036940,-95.682870,"Sw 17th St & Sw Topeka Blvd Topeka, KS, 66612,...",2022-02-09T12:37:11-05:00,2022-02-09T12:37:17-05:00,None,https://seeclickfix.com/api/v2/issues/11852838,"{'video_url': None, 'image_full': None, 'image..."
4,11852731,Acknowledged,Police General Issue,"Silver Subaru Outback. Parked on the street, w...",39.045188,-95.709334,"1218 Sw Medford Ave Topeka, KS, 66604, USA",2022-02-09T12:21:41-05:00,2022-02-09T12:21:46-05:00,None,https://seeclickfix.com/api/v2/issues/11852731,"{'video_url': None, 'image_full': None, 'image..."
5,11852433,Acknowledged,Loud pedestrian crossing,The traffic light at 101 SE 9th St has an aud...,39.047363,-95.675149,"Se 9th St & S Kansas Ave Topeka, KS, 66612, USA",2022-02-09T11:38:19-05:00,2022-02-09T11:38:23-05:00,None,https://seeclickfix.com/api/v2/issues/11852433,"{'video_url': None, 'image_full': None, 'image..."
6,11851623,Acknowledged,Utility Work Damage,This is the sewage line to their house that ha...,39.049810,-95.706400,"1041 Sw Macvicar Ave Topeka, KS, 66604, USA",2022-02-09T09:24:39-05:00,2022-02-09T09:25:16-05:00,None,https://seeclickfix.com/api/v2/issues/11851623,"{'video_url': None, 'image_full': 'https://see..."
7,11849500,Acknowledged,Police General Issue,Cars and trailers parked in yard and not on a ...,39.045887,-95.645804,"540 Se Burr St Topeka KS 66607, United States",2022-02-08T19:09:26-05:00,2022-02-08T19:09:29-05:00,None,https://seeclickfix.com/api/v2/issues/11849500,"{'video_url': None, 'image_full': None, 'image..."
8,11847298,Closed,Hydrant Leaking,The fire hydrant located on the SW corner of 1...,39.051260,-95.707397,"2401 Sw 10th Ave Topeka, KS, 66604, USA",2022-02-08T12:46:29-05:00,2022-02-08T12:46:33-05:00,2022-02-11T12:33:23-05:00,https://seeclickfix.com/api/v2/issues/11847298,"{'video_url': None, 'image_full': None, 'image..."
9,11846696,Closed,Pothole,A pothole has been slowly growing every since ...,39.048663,-95.673961,"100-198 Se 8th Ave Topeka, KS, 66603, USA",2022-02-08T11:17:29-05:00,2022-02-08T11:17:34-05:00,2022-02-09T15:05:11-05:00,https://seeclickfix.com/api/v2/issues/11846696,"{'video_url': None, 'image_full': None, 'image..."
